In [75]:
import pandas as pd
import numpy as np

df = pd.read_csv('packages.csv')

df.columns

Index(['keywords', 'Days', 'fb id'], dtype='object')

In [76]:
#We can compute the similarity between travel descriptions using TfidfVectorizer

#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

#Replace NaN with an empty string
df["keywords"] = df["keywords"].fillna("")

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(df["keywords"])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(6, 20)

In [77]:
# Import linear_kernel, cosine_similarity, and sigmoid_kernel
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import sigmoid_kernel

# Compute the cosine similarity matrix
linear = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
sig_score = sigmoid_kernel(tfidf_matrix, tfidf_matrix)

In [78]:
#Construct a reverse map of indices and product names
indices = pd.Series(df.index, index=df["keywords"])

In [79]:
import pickle

with open("linear_model_plan.pkl" , 'wb') as f:
    pickle.dump(linear, f)

In [62]:
with open("text_vectorizer_plan.pkl" ):
pickle.dump(f)

SyntaxError: invalid syntax (2015104495.py, line 1)

In [80]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

choices = list(indices.index)
extracted = process.extract("lego", choices, limit=1)
extracted[0][0]

'Historical, Buddha Statues, Religious, Cultural'

In [86]:
# Function that takes in prompt as input and outputs most similar product


def rec_lin(user_input,days, linear=linear):
    
    # use fuzzywuzzy to grab the product with name closest to user input
    extracted = process.extract(user_input, choices, limit=1)
    destination = extracted[0][0]
    
    # Get the index of the product that matches the product name
    idx = indices[destination]

    # Get the pairwise similarity scores
    sim_scores = list(enumerate(linear[idx]))

    # Sort the products based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar products
    sim_scores = sim_scores[1:11]

    # Get the product indices
    product_indices = [i[0] for i in sim_scores]

    df_return = df[["Days", "fb id"]].loc[product_indices]

    df_return_list  = df_return[df_return["Days"]==days]["fb id"].to_list()
    
    # Return the top 10 most similar products
    return df_return_list

In [ ]:
rec_lin("Hitorical", 3)